In [ ]:
# Until v4.26.0 is out, requires:
# pip install --upgrade git+https://github.com/huggingface/transformers
# Sometimes, I may have to use bleeding edge:
# pip install -U git+https://github.com/huggingface/diffusers

import time

import IPython
import numpy as np

import sdcommon
import depth2img


def run(save=True, **kwargs):
    ml = depth2img.ML()
    params = depth2img.Params(**kwargs)
    start = time.time()
    imgs, depth = ml.run(params) #, local_files_only=False)
    print("Took %.1fs" % (time.time()-start))
    if save:
      print("Saved as %s.png" % sdcommon.save(ml, params, depth, *imgs)) 
    return imgs, depth

for seed in range(1, 5):
    for strength in np.arange(0.3, 0.8, 0.1):
        imgs, _ = run(
            imagename="out/txt2img00000.png",
            prompt="TODO",
            steps=50,
            strength=strength,
            seed=seed)
        print("seed:", seed, "strength:", strength)
        IPython.display.display(imgs[0])